In [74]:
import numpy as np
import pandas as pd
#from google.colab import drive
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import time

In [46]:
#dealing with data

df = pd.read_csv('Measured_data.csv')
wavelength = pd.read_csv('Vis-NIR_Reflectance.csv')

df = category.merge(wavelength, 'left') #combine with wavelength data

for i in range(len(df)):
    df["ID"][i] = df["ID"][i].split("-",1)[0]

##coordinate data
coordinates = pd.read_csv("coordinates.csv", header=None)
coordinates.columns = coordinates.iloc[0,:]
coordinates = coordinates.iloc[1:, :]
coordinates.iloc[:,1:] = coordinates.iloc[:,1:].astype(float)
    

data = df.merge(coordinates, on='ID', how='left')


/var/folders/dp/g8s606fx4r30xp70s3n58xwr0000gn/T/ipykernel_89239/3612199851.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ID"][i] = df["ID"][i].split("-",1)[0]


,ID,Depth,pH,Clay (%),Silt (%),Sand (%),SOC (%),N (%),350,351,...,2493,2494,2495,2496,2497,2498,2499,2500,lat,lon
0,OB2,0-10,7.34,18,39,43,1.874400,0.190490,18.438100,18.199500,...,46.287000,46.243700,46.245833,46.272833,46.302433,46.338800,46.377267,46.418400,42.88705,-89.421793
1,OB2,10-20,7.44,16,42,42,1.333280,0.163019,18.125900,17.772867,...,40.779400,40.710000,40.675967,40.660267,40.657533,40.666367,40.664833,40.653467,42.88705,-89.421793
2,OB2,20-30,7.40,33,30,36,1.041307,0.108395,17.342200,17.121467,...,35.133100,35.089700,35.069733,35.067067,35.094100,35.143433,35.176300,35.194333,42.88705,-89.421793
3,OB2,30-40,7.68,37,18,45,4.936881,0.135670,19.004733,18.644967,...,36.663800,36.536100,36.431900,36.343233,36.280367,36.238267,36.192600,36.144633,42.88705,-89.421793
4,OB2,40-50,7.84,5,28,66,10.405942,0.063240,24.988100,24.574033,...,49.104567,48.918200,48.778433,48.664767,48.549667,48.439867,48.341467,48.254733,42.88705,-89.421793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,OB100,30-40,6.27,23,22,55,0.384374,0.019708,18.679800,18.453400,...,33.826200,33.784733,33.730833,33.672133,33.614967,33.560567,33.532400,33.530667,42.860924,-89.404809
193,OB100,40-50,6.55,20,17,63,0.243897,0.013222,22.507433,22.223467,...,38.529533,38.461000,38.355300,38.240067,38.169433,38.130200,38.111200,38.114267,42.860924,-89.404809
194,OB100,50-60,6.70,21,24,55,0.220232,0.038898,20.264200,20.063800,...,38.905500,38.851200,38.763500,38.665133,38.600400,38.558833,38.535567,38.532200,42.860924,-89.404809
195,OB100,60-70,6.68,30,53,17,0.212419,0.037179,30.855433,30.692567,...,52.462433,52.373700,52.310867,52.264200,52.231733,52.212733,52.198767,52.190400,42.860924,-89.404809


In [64]:
#train-test split
train = df[df['Depth'] == '0-10'] ## possibly train on different depths 
temp = df[df['Depth'] != '0-10']

X_train = train.iloc[:, [3,4,5,6,7]] ## todo add more predictors (wavelength, lat, long)
y_train = train.iloc[:, 2]

X_tmp = temp .iloc[:, [3,4,5,6,7]]
y_tmp = temp .iloc[:, 2]
X_valid, X_test, y_valid, y_test = train_test_split(X_tmp, y_tmp, test_size=.5, random_state=0)



In [52]:
#classifiers we want to test

classifiers = [GradientBoostingRegressor(random_state=0), RandomForestRegressor(), KNeighborsRegressor()]
gradientParameters = {'n_estimators': [10, 50, 100, 150], 'learning_rate': [0.01, 0.25, 1, 1.3], 'max_depth': [1,2]}
RandomForestParameters = {'n_estimators': [10,50,100,200],'max_depth': [1, 2, 3, 4, 5]}
KNNParameters = {'n_neighbors': [1, 2, 3, 4]}
parameters = [gradientParameters, RandomForestParameters, KNNParameters]

In [75]:
#train models on the training data (soil at depth "0-10")

times = list()

bestClassifierIndex = -1
bestMSE = np.Inf
bestClassifier = None
for i in range(0, 3):
    print(i, "...", sep="")
    start = time.time()
    clf = GridSearchCV(classifiers[i], parameters[i])
    clf.fit(X_train, y_train)
    end = time.time()
    diff = end - start
    times.append(diff)

    score = mean_squared_error(y_valid, clf.predict(X_valid))
    if score < bestMSE:
        bestClassifierIndex = i
        bestClassifier = clf
        bestMSE = score
        
#show computation times of each respective model
print(times)

0...
1...
2...
[2.0102527141571045, 6.744227886199951, 0.062274932861328125]


In [73]:
print("Best MSE: ", bestMSE)
print("Best Classifier Index: ", bestClassifierIndex)
print("Best Classifier: ", bestClassifier)
bestGuess = list(bestClassifier.predict(X_test))
y_test = list(y_test)
correct = 0
num_iterations = len(y_test)
for i in range(num_iterations):
    if abs(bestGuess[i] - y_test[i]) < 0.55: ##if the difference is between 0.55 we say the prediction was correct
        print(bestGuess[i] )
        print(y_test[i] )
        print(i)
        correct = correct + 1
        
print(correct, f" out of {num_iterations} are correct.") #we'll probably want 

Best MSE:  0.4711747589046685
Best Classifier Index:  0
Best Classifier:  GridSearchCV(estimator=GradientBoostingRegressor(random_state=0),
             param_grid={'learning_rate': [0.01, 0.25, 1, 1.3],
                         'max_depth': [1, 2],
                         'n_estimators': [10, 50, 100, 150]})
6.516430501218329
6.43
0
6.516430501218329
6.64
2
6.516430501218329
6.45
3
6.516430501218329
6.54
5
5.88305102219833
5.98
7
6.516430501218329
6.62
9
6.516430501218329
6.88
13
6.516430501218329
6.15
14
6.45304633165102
6.85
15
6.45304633165102
6.16
16
6.516430501218329
6.87
17
5.88305102219833
6.43
18
6.516430501218329
6.49
19
6.516430501218329
6.5
20
6.516430501218329
6.89
21
6.516430501218329
6.46
23
6.014354805931905
6.41
24
6.516430501218329
6.92
25
6.516430501218329
6.86
26
6.516430501218329
6.79
28
6.516430501218329
6.32
29
5.88305102219833
5.88
30
6.516430501218329
6.34
31
6.516430501218329
6.76
32
6.516430501218329
6.37
33
6.516430501218329
6.58
35
6.516430501218329
6.23
3